In [19]:
import requests
import json
from tqdm import tqdm

In [20]:
genes = ['TSC22D1',
 'KLF1',
 'MAP2K6',
 'CEBPE',
 'RUNX1T1',
 'MAML2',
 'CBL',
 'PTPN9',
 'TGFBR2',
 'ETS2',
 'SGK1',
 'TBX3',
 'DUSP9',
 'SPI1',
 'ELMSAN1',
 'UBASH3B',
 'PTPN12',
 'FOXA1',
 'FOXA3',
 'IGDCC3',
 'BCORL1',
 'MEIS1',
 'GLB1L2',
 'IKZF3',
 'BAK1',
 'FEV',
 'MAP2K3',
 'SLC38A2',
 'SET',
 'LHX1',
 'TBX2',
 'SLC4A1',
 'RREB1',
 'ZNF318',
 'MAPK1',
 'COL2A1',
 'ZBTB25',
 'MAP4K5',
 'SLC6A9',
 'MIDN',
 'OSR2',
 'DLX2',
 'CBFA2T3',
 'HES7',
 'FOXL2',
 'AHR',
 'FOXO4',
 'RHOXF2BB',
 'S1PR2',
 'POU3F2',
 'LYL1',
 'IER5L',
 'CNN1',
 'CELF2',
 'JUN',
 'CEBPA',
 'MAP4K3',
 'ZC3HAV1',
 'CDKN1A',
 'UBASH3A',
 'PRTG',
 'PTPN1',
 'TP73',
 'MAP7D1',
 'FOSB',
 'C19orf26',
 'IRF1',
 'TMSB4X',
 'BPGM',
 'SAMD1',
 'HOXB9',
 'HOXC13',
 'CKS1B',
 'CLDN6',
 'KIF18B',
 'KIF2C',
 'BCL2L11',
 'COL1A1',
 'CEBPB',
 'FOXF1',
 'ZBTB1',
 'PLK4',
 'ARRDC3',
 'C3orf72',
 'KIAA1804',
 'HNF4A',
 'SNAI1',
 'KMT2A',
 'ISL2',
 'CSRNP1',
 'ARID1A',
 'CNNM4',
 'NCL',
 'ZBTB10',
 'STIL',
 'ATL1',
 'NIT1',
 'CDKN1B',
 'PTPN13',
 'HOXA13',
 'CITED1',
 'PRDM1',
 'HK2',
 'CDKN1C',
 'EGR1']

In [21]:
result = {}
for gene in tqdm(genes):
    gene_code = gene
    headers = {
        'accept': 'application/json',
    }
    
    params = {
        'interactors': 'true',
        'species': 'Homo sapiens',
        'pageSize': '20',
        'page': '1',
        'sortBy': 'ENTITIES_PVALUE',
        'order': 'ASC',
        'resource': 'TOTAL',
        'pValue': '1',
        'includeDisease': 'true',
    }
    
    response = requests.get(f'https://reactome.org/AnalysisService/identifier/{gene_code}/projection', params=params, headers=headers)
    if response.status_code != 200:
        print(response.status_code)
    else:
        r = json.loads(response.content.decode())
        result[gene] = r    
    

100%|██████████████████████████████████████████████████████████████████| 105/105 [02:08<00:00,  1.23s/it]


In [22]:
with open('reactome_genes.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)

In [9]:
r.keys()

dict_keys(['summary', 'expression', 'identifiersNotFound', 'pathwaysFound', 'pathways', 'resourceSummary', 'speciesSummary', 'warnings'])

In [10]:
r['pathwaysFound']

553

In [11]:
r['pathways']

[{'stId': 'R-HSA-9723907',
  'dbId': 9723907,
  'name': 'Loss of Function of TP53 in Cancer',
  'species': {'dbId': 48887, 'taxId': '9606', 'name': 'Homo sapiens'},
  'llp': False,
  'entities': {'resource': 'TOTAL',
   'total': 1,
   'found': 1,
   'ratio': 4.1825254088418586e-05,
   'curatedTotal': 1,
   'curatedFound': 1,
   'interactorsTotal': 0,
   'interactorsFound': 0,
   'pValue': 0.0002091087776552314,
   'fdr': 0.06105976307532757,
   'exp': []},
  'reactions': {'resource': 'TOTAL',
   'total': 1,
   'found': 1,
   'ratio': 6.666666666666667e-05},
  'inDisease': True},
 {'stId': 'R-HSA-9723905',
  'dbId': 9723905,
  'name': 'Loss of function of TP53 in cancer due to loss of tetramerization ability',
  'species': {'dbId': 48887, 'taxId': '9606', 'name': 'Homo sapiens'},
  'llp': True,
  'entities': {'resource': 'TOTAL',
   'total': 1,
   'found': 1,
   'ratio': 4.1825254088418586e-05,
   'curatedTotal': 1,
   'curatedFound': 1,
   'interactorsTotal': 0,
   'interactorsFound': 